# Testy AutoMappera

## Motywacja

Sprawdzić czy ProjectTo<> ma wpływ na na budowanie query, aby dowieść, że Queries są niepotrzebne w naszym projekcie w większości przypadków

## Przygotowanie
Tutaj importuję wszystkie niezbędne biblioteki oraz deklaruje klasy pomocnicze.

In [ ]:
using System.Linq;
using System.Collections.Generic;

### EntityFramework

In [ ]:
#r "nuget:Microsoft.EntityFrameworkCore,6.0.6"
#r "nuget:Microsoft.EntityFrameworkCore.Sqlite.Core,6.0.6"

Installed Packages Microsoft.EntityFrameworkCore, 6.0.6 Microsoft.EntityFrameworkCore.Sqlite.Core, 6.0.6

In [ ]:
using Microsoft.EntityFrameworkCore;

In [ ]:
public class Article
{
    public int ArticleId { get; set; }
    public string Articlenumber { get; set; }
}

public class Articlewarehouse
{
    public int ArticlewarehouseId { get; set; }
    public int ArticleId { get; set; }

    public Article Article { get; set; }
}

public class MyDbContext: DbContext
{
    public DbSet<Article> Articles { get; set; }
    public DbSet<Articlewarehouse> Articlewarehouses { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        optionsBuilder.UseSqlite("Data Source=sample.db");
    }
}

### AutoMapper

In [ ]:
#r "nuget:AutoMapper,12.0.1"

Installed Packages AutoMapper, 12.0.1

In [ ]:
using AutoMapper;
using AutoMapper.QueryableExtensions;

### Unit test

In [ ]:
#r "nuget:xunit,2.4.2"
#r "nuget:FluentAssertions,6.9.0"
#r "nuget:NSubstitute,4.4.0"

Installed Packages FluentAssertions, 6.9.0 NSubstitute, 4.4.0 xunit, 2.4.2

In [ ]:
using Xunit;
using Xunit.Sdk;

using FluentAssertions;

using NSubstitute;

In [ ]:
// Test executor

public void RunTest(Action action)
{
    try
    {
        action();

        $"{action.Method.Name}: Success".Display();
    }
    catch(XunitException)
    {
        $"{action.Method.Name}: Fail".Display();
    }
    catch(Exception exeption)
    {
        $"{action.Method.Name}: Fail".Display();
        throw exeption;
    }
}

### Konfiguracja AutoMappera oraz DTO

In [ ]:
public class ArticleDto
{
    public int ArticleId { get; set; }
    public string Articlenumber { get; set; }
}

In [ ]:
public class ArticleWarehouseWithoutArticleDto
{
    public int ArticlewarehouseId { get; set; }
    public int ArticleId { get; set; }
}

In [ ]:
public class ArticleWarehouseDto
{
    public int ArticlewarehouseId { get; set; }
    public int ArticleId { get; set; }

    public ArticleDto Article { get; set; }
}

In [ ]:
var configuration = new MapperConfiguration(cfg => 
    {
        cfg.CreateMap<Article, ArticleDto>();
        cfg.CreateMap<Articlewarehouse, ArticleWarehouseDto>();
        cfg.CreateMap<Articlewarehouse, ArticleWarehouseWithoutArticleDto>();
    });

var mapper = configuration.CreateMapper();

## Analiza

### Wpływ ProjectTo<> na liczbę kolumn w SELECT

In [ ]:
using(var dbContext = new MyDbContext())
{
    dbContext.Articles.ProjectTo<ArticleDto>(configuration).ToQueryString().Display();
}

SELECT "a"."ArticleId", "a"."Articlenumber"
FROM "Articles" AS "a"

``` sql
SELECT "a"."ArticleId", "a"."Articlenumber"
FROM "Articles" AS "a"
```

Jak widać na powyższym ProjectTo<> ogarniacza liczbę kolumn bazując na polach, które udało się zmapować.

### Wpływ ProjectTo<> na JOIN

In [ ]:
using(var dbContext = new MyDbContext())
{
    dbContext.Articlewarehouses.ProjectTo<ArticleWarehouseDto>(configuration)
        .ToQueryString()
        .Display();

    dbContext.Articlewarehouses.ProjectTo<ArticleWarehouseWithoutArticleDto>(configuration)
        .ToQueryString()
        .Display();

    dbContext.Articlewarehouses
        .Include(x => x.Article)
        .ProjectTo<ArticleWarehouseWithoutArticleDto>(configuration)
        .ToQueryString()
        .Display();

    dbContext.Articlewarehouses
        .Where(x => x.Article.Articlenumber == "000000000000")
        .ProjectTo<ArticleWarehouseWithoutArticleDto>(configuration)
        .ToQueryString()
        .Display();
}

SELECT "a"."ArticlewarehouseId", "a"."ArticleId", 0, "a0"."ArticleId", "a0"."Articlenumber"
FROM "Articlewarehouses" AS "a"
INNER JOIN "Articles" AS "a0" ON "a"."ArticleId" = "a0"."ArticleId"

SELECT "a"."ArticlewarehouseId", "a"."ArticleId"
FROM "Articlewarehouses" AS "a"

SELECT "a"."ArticlewarehouseId", "a"."ArticleId"
FROM "Articlewarehouses" AS "a"

SELECT "a"."ArticlewarehouseId", "a"."ArticleId"
FROM "Articlewarehouses" AS "a"
INNER JOIN "Articles" AS "a0" ON "a"."ArticleId" = "a0"."ArticleId"
WHERE "a0"."Articlenumber" = '000000000000'

``` sql
SELECT "a"."ArticlewarehouseId", "a"."ArticleId", 0, "a0"."ArticleId", "a0"."Articlenumber"
FROM "Articlewarehouses" AS "a"
INNER JOIN "Articles" AS "a0" ON "a"."ArticleId" = "a0"."ArticleId"

SELECT "a"."ArticlewarehouseId", "a"."ArticleId"
FROM "Articlewarehouses" AS "a"

SELECT "a"."ArticlewarehouseId", "a"."ArticleId"
FROM "Articlewarehouses" AS "a"

SELECT "a"."ArticlewarehouseId", "a"."ArticleId"
FROM "Articlewarehouses" AS "a"
INNER JOIN "Articles" AS "a0" ON "a"."ArticleId" = "a0"."ArticleId"
WHERE "a0"."Articlenumber" = '000000000000'
```

Jak widać na powyższym przykładzie AutoMapper w procesie mapowanie pozbywa się lub dodaje JOINy według potrzeby. Tak więc nie ma konieczności w takim wypadku wykorzystywania metody .Include(...)